<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/confirm_label_noize_for_cm_data/eras/confirm_label/examine_KTH_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 実験条件（外乱）を定める

In [ ]:
LABEL_NOISE_RATE = 0
TEST_DATA_RATE = 0.3

#学習条件を定める

##学習手法の仕様

In [ ]:
learningDict = {
    "optimizer" : {
        "this.optimizer" : "sgd",
        "learning_rate" : 1e-5,
        "momentum" : 0,
        "decay" : 0.1
        "nesterov" : False
    },
    "theWay" : {
        "batch_size" : 4,
        "epochs" : 1024,
    },
    "compared_losses" : [
                        #  tf.keras.losses.CategoricalCrossentropy(),
                        #  tf.keras.losses.MeanSquaredError(), 
                         tf.keras.losses.MeanAbsoluteError(),
                        #  tf.keras.losses.SquaredHinge()               
    ]
}
def compile_optimizer():
  # 最適化処理 (adamのみ対応)
  if learningDict["optimizer"]["this.optimizer"] == "adam":
    optimizer = keras.optimizers.Adam(
        lr=learningDict["optimizer"]["learning_rate"],
        beta_1=learningDict["optimizer"]["beta_1"],
        beta_2=learningDict["optimizer"]["beta_2"])
    print("adam is used as a optimizer")

  elif learningDict["optimizer"]["this.optimizer"] == "Nadam":
    optimizer = keras.optimizers.Nadam(
        lr=learningDict["optimizer"]["learning_rate"],
        beta_1=learningDict["optimizer"]["beta_1"],
        beta_2=learningDict["optimizer"]["beta_2"],
        epsilon=None, 
        schedule_decay=0.4)
    print("Nadam is used as a optimizer")

  elif learningDict["optimizer"]["this.optimizer"] == "sgd":
    optimizer = keras.optimizers.SGD(
        lr=learningDict["optimizer"]["learning_rate"],
        momentum=learningDict["optimizer"]["momentum"],
        decay=learningDict["optimizer"]["decay"],
        nesterov=learningDict["optimizer"]["nesterov"]) 
    print("sgd is used as a optimizer")
  else:
    print("error")
  
  return optimizer

## モデルの仕様（ニューラルネットワーク）
<ul>
  <li>入力層(フレームサイズ, フレームの高さ, フレームの横幅, RGB情報) </li>
  <li>出力層(予測値) </li>
  <li> 中間層 
    <ol>
      <li>conv0</li>
      <li>pool0</li>
      <li>conv1</li>
      <li>pool1</li>
      <li>dence0</li>
  </li>
</ui>

In [5]:
def make_model(video_format):
  # モデル作成
  model = models.Sequential()
  # 入力層
  model.add(
      layers.Reshape(
          (video_format.FRAME_SIZE,
          video_format.HEIGHT,
          video_format.WIDTH,
          video_format.COLORinfo),
          input_shape=(video_format.FRAME_SIZE * video_format.HEIGHT * video_format.WIDTH * video_format.COLORinfo,),
          name='Input_Layer' )
  )
  # 畳み込み0
  model.add(
      layers.Conv3D(
          filters=32,
          kernel_size=(3, 3, 3),
          strides=(1, 1, 1),
          padding='same',
          activation='relu',
          name='conv0'))
  # pool0
  model.add(
      layers.MaxPooling3D(pool_size=(2, 2, 2), name='pool0'))

  # 畳み込み1
  model.add(
      layers.Conv3D(
          filters=32,
          kernel_size=(3, 3, 3),
          strides=(1, 1, 1),
          padding='same',
          activation='relu',
          name='conv1'))
  # pool1
  model.add(
      layers.MaxPooling3D(pool_size=(2, 2, 2), name='pool1'))

  # dence
  model.add(
      layers.Dense(1024,
        activation='relu',
        name='dence0' ),
  )
  # 出力層
  model.add(
      layers.Dense(4, activation='softmax', name='WATERSUPPLY')
  )
  return model


##データの仕様

In [ ]:
## フォーマットの設定
class video_format:
  # 想定された入力CMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から計算される値